# N-gram Language Models

N-gram language models are statistical models used to predict the likelihood of a sequence of words in a language. They work by estimating the probability of a word based on the previous $n-1$ words in the sequence.

## N-gram Model
The probability of a sentence $W = w_1, w_2, \dots, w_m$ is approximated using the Markov assumption, which states that the probability of a word depends only on the previous $n-1$ words:

$$
P(w_1, w_2, \dots, w_m) \approx \prod_{i=1}^{m} P(w_i \mid w_{i-n+1}, \dots, w_{i-1})
$$

For instance, in a unigram model ($n=1$), the probability of each word is independent of the others. In a bigram model ($n=2$), the probability of each word depends on the previous word.

In [46]:
import datasets
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm


train_data, test_data = datasets.load_dataset("imdb", split=["train", "test"])

In [47]:
train_text=[sample['text'] for sample in train_data]
train_text[0]

'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, ev

## Unigram Model
In the unigram model, each word is treated independently. The probability of a sentence is given by:

$$
P(w_1, w_2, \dots, w_m) = \prod_{i=1}^{m} P(w_i)
$$

To account for unseen words, we apply **Laplace smoothing**:

$$
P(w_i) = \frac{\text{count}(w_i) + 1}{N + V}
$$

where:
- $\text{count}(w_i)$ is the frequency of word $w_i$,
- $N$ is the total number of words in the corpus,
- $V$ is the vocabulary size.

In [48]:
from collections import Counter, defaultdict
count_unigram=Counter()
for text in tqdm(train_text):
    tokens=text.lower().split()
    for word, count in Counter(tokens).items():
        count_unigram[word]+=count

100%|████████████████████████████████████████| 25000/25000 [00:02<00:00, 11346.57it/s]


In [51]:
proba_unigram=Counter()
N=np.sum(list(count_unigram.values()))
for word in count_unigram:
    proba_unigram[word]=(count_unigram[word]+1)/(N+len(count_unigram))

In [52]:
proba_unigram

Counter({'the': 0.05285141832355503,
         'a': 0.02623780882785459,
         'and': 0.02601127861297239,
         'of': 0.02369676642471184,
         'to': 0.021975235211686008,
         'is': 0.017087694094647637,
         'in': 0.014849621500981658,
         'i': 0.011561242632231888,
         'this': 0.011435592998198749,
         'that': 0.010874270481669506,
         'it': 0.01074517614487567,
         '/><br': 0.008355208562809316,
         'was': 0.007713673681995211,
         'as': 0.007398401362658799,
         'for': 0.007027849765686397,
         'with': 0.007009149950699742,
         'but': 0.006522790727581915,
         'on': 0.00518673815682485,
         'movie': 0.00506666566059475,
         'his': 0.0047668124869490874,
         'are': 0.004714977912073798,
         'not': 0.0046910290262136955,
         'film': 0.004556521585081616,
         'you': 0.004521582457080234,
         'have': 0.004485495094825286,
         'he': 0.004294068041409264,
         'be': 0.004

In [61]:

text=train_text[0]
prob_sent=0
tokens=text.lower().split()
for word, count in Counter(tokens).items():
        prob_sent+=count*np.log(proba_unigram[word])
    
print('Sentence Log Proba :',prob_sent)

Sentence Log Proba : -2151.8445716329484


### Generating Sentences
Sentences are generated by sampling words from the unigram distribution until a stopping condition (e.g., sampling a special end-of-sentence token).

In [65]:
max_generation=10
generated_sent=''
for it in range(max_generation):
    next_word=np.random.choice(list(proba_unigram.keys()), size=1, p=list(proba_unigram.values()))[0]
    generated_sent+=next_word+' '
print('Generated text :', generated_sent)

Generated text : second older/bigger; ok to statement (douglas you. casino that terrorism 


## Bigram Model
In the bigram model, the probability of a word depends on the preceding word. The probability of a sentence is:

$$
P(w_1, w_2, \dots, w_m) = P(w_1) \prod_{i=2}^{m} P(w_i \mid w_{i-1})
$$

With Laplace smoothing, the conditional probability is:

$$
P(w_i \mid w_{i-1}) = \frac{\text{count}(w_{i-1}, w_i) + 1}{\text{count}(w_{i-1}) + V}
$$



In [70]:
import nltk
count_context=Counter()
context_ngram_count=defaultdict(lambda : Counter())

for text in tqdm(train_text):
    tokens=['<bos>']+text.lower().split()+['<eos>']
    for ngram in nltk.ngrams(tokens, 2):
        context=ngram[:-1]
        word=ngram[-1]
        count_context[context]+=1
        context_ngram_count[context][word]+=1

100%|█████████████████████████████████████████| 25000/25000 [00:09<00:00, 2506.81it/s]


In [74]:
prob_bigram=defaultdict(lambda : Counter())
for context in tqdm(count_context):
    N=np.sum(list(context_ngram_count[context].values()))
    for word in context_ngram_count[context]:
        prob_bigram[context][word]=(context_ngram_count[context][word]+1)/(N+len(count_context))

100%|██████████████████████████████████████| 251638/251638 [00:04<00:00, 60809.08it/s]


In [76]:
context, word

(('crocker)',), 'are')

In [75]:
prob_bigram[context][word]

7.947893609496143e-06

In [78]:
text=train_text[0]
tokens=['<bos>']+text.lower().split()+['<eos>']
prob_sent=0
for ngram in nltk.ngrams(tokens, 2):
        context=ngram[:-1]
        word=ngram[-1]
        prob_sent+=np.log(prob_bigram[context][word])
print('Proba Sent :', prob_sent)

Proba Sent : -2600.039088218897


### Generating Sentences
Sentences are generated by selecting the first word from $P(w_1)$, then iteratively sampling each subsequent word based on the conditional probability $P(w_i \mid w_{i-1})$ until a stopping condition is met.

In [93]:
max_length=20



In [98]:
current_context

'snow'

In [99]:
list(prob_bigram[current_context].keys())

[]

In [103]:
current_context='<bos>'
generated_sent=''
for it in range(max_length):
    proba=list(prob_bigram[(current_context,)].values())
    proba/=np.sum(proba)
    current_context=np.random.choice(list(prob_bigram[(current_context,)].keys()), size=1, p=proba)[0]
    if  current_context=='<eos>':
        break
    generated_sent+=current_context+' '

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [105]:
print('Generated Text :', generated_sent)

Generated Text : superb as pyramids is simple lives. shakespeare work with. although carlton-browne revels in 1999 cannes and ava gardner, horror of 



## Perplexity
Perplexity measures the quality of a language model. It is the inverse of the geometric mean of the probabilities assigned to a test set:

$$
\text{Perplexity}(W) = 2^{-\frac{1}{N} \sum_{i=1}^{N} \log_2 P(w_i \mid w_{i-n+1}, \dots, w_{i-1})}
$$

where $N$ is the number of words in the test set, and $P(w_i \mid w_{i-n+1}, \dots, w_{i-1})$ is the model's predicted probability for word $w_i$.

Lower perplexity indicates a better model. It reflects how well the model predicts the test data.




In [114]:
text=train_text[0]
tokens=['<bos>']+text.lower().split()+['<eos>']
prob_sent=0
for ngram in nltk.ngrams(tokens, 2):
        context=ngram[:-1]
        word=ngram[-1]
        prob_sent+=np.log(prob_bigram[context][word])
print('Perplexity:', 2**(-prob_sent/len(list(nltk.ngrams(tokens, 2)))))

Perplexity: 510.8213611885138
